In [1]:
# How good are Small LLMs at chess gameplay?
# Can they explain their reasoning?
# Does finetuning make them better?
# Do reasoning (e.g. chain-of-thought) techniques improve their performance?
# Can they be finetuned via RL selfplay?

#### Part 1: Eval off-the-shelf LLMs on chess gameplay

Use huggingface LLM api and python-chess and stockfish api to evaluate LLMs on chess gameplay.

In [2]:
# install python-chess and stockfish if not already installed

In [3]:
try:
  from stockfish import Stockfish
except ImportError:
  !pip install stockfish
  from stockfish import Stockfish

try:
  import chess
except ImportError:
  !pip install python-chess
  import chess

In [4]:
import os

In [5]:
STOCKFISH_REMOTE = "https://github.com/official-stockfish/Stockfish/releases/latest/download/stockfish-ubuntu-x86-64-avx2.tar"
STOCKFISH_LOCAL = "./stockfish/stockfish-ubuntu-x86-64-avx2"

In [6]:

if not os.path.exists(STOCKFISH_LOCAL):
  !wget -O stockfish.tar $STOCKFISH_REMOTE
  !tar -xf stockfish.tar
  !rm stockfish.tar

In [7]:
os.path.exists(STOCKFISH_LOCAL)

True

In [8]:
from stockfish import Stockfish

stockfish_params = {
    # "Debug Log File": "",
    "Contempt": 0,
    "Min Split Depth": 0,
    "Threads": 1, # More threads will make the engine stronger, but should be kept at less than the number of logical processors on your computer.
    "Ponder": "false",
    "Hash": 256, # Default size is 16 MB. It's recommended that you increase this value, but keep it as some power of 2. E.g., if you're fine using 2 GB of RAM, set Hash to 2048 (11th power of 2).
    "MultiPV": 1,
    "Skill Level": 20,
    "Move Overhead": 10,
    "Minimum Thinking Time": 20,
    "Slow Mover": 100,
    "UCI_Chess960": "false",
    "UCI_LimitStrength": "false",
    "UCI_Elo": 1350
}

stockfish = Stockfish(STOCKFISH_LOCAL, parameters=stockfish_params)

In [9]:
print(stockfish.get_board_visual())
stockfish.get_best_move()

+---+---+---+---+---+---+---+---+
| r | n | b | q | k | b | n | r | 8
+---+---+---+---+---+---+---+---+
| p | p | p | p | p | p | p | p | 7
+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   | 6
+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   | 5
+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   | 4
+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   | 3
+---+---+---+---+---+---+---+---+
| P | P | P | P | P | P | P | P | 2
+---+---+---+---+---+---+---+---+
| R | N | B | Q | K | B | N | R | 1
+---+---+---+---+---+---+---+---+
  a   b   c   d   e   f   g   h



'e2e4'

In [10]:
stockfish.get_fen_position()

'rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1'

In [11]:
import chess

board = chess.Board()

print(board)

r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
P P P P P P P P
R N B Q K B N R


In [12]:
stockfish.set_fen_position(board.fen(), send_ucinewgame_token=True)
best_move = stockfish.get_best_move()
board.push_uci(best_move)
print(board)

r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . P . . .
. . . . . . . .
P P P P . P P P
R N B Q K B N R


In [13]:
best_move

'e2e4'

In [14]:
board.outcome()

In [15]:
# huggingface AutoModelForCausalLM
try:
  from transformers import AutoModelForCausalLM, AutoTokenizer
except ImportError:
  !pip install transformers
  from transformers import AutoModelForCausalLM, AutoTokenizer

/usr/local/lib/python3.11/dist-packages/torch_xla/__init__.py:253: UserWarning: `tensorflow` can conflict with `torch-xla`. Prefer `tensorflow-cpu` when using PyTorch/XLA. To silence this warning, `pip uninstall -y tensorflow && pip install tensorflow-cpu`. If you are in a notebook environment such as Colab or Kaggle, restart your notebook runtime afterwards.
  warnings.warn(


In [ ]:
import torch
from transformers import pipeline

model_id = "meta-llama/Llama-3.2-3B-Instruct"
pipe = pipeline(
    "text-generation",
    model=model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)


In [27]:
BASE_PROMPT = """You are a AI chess engine. You are playing as {llm_color}, and your opponent is {opponent_color}. It is your turn. The board is as follows:
{board}
The sequence of moves so far is: [{history}]
Select your next moves in UCI notation. Only return your move and not additional text."""
prompt = BASE_PROMPT.format(llm_color="black", opponent_color="white", board=board, history=f"{best_move}")
print(prompt)

You are a AI chess engine. You are playing as black, and your opponent is white. It is your turn. The board is as follows:
r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . P . . .
. . . . . . . .
P P P P . P P P
R N B Q K B N R
The sequence of moves so far is: [e2e4]
Select your next moves in UCI notation. Only return your move and not additional text.


In [25]:
messages = [
    {"role": "system", "content": prompt},
    {"role": "user", "content": "Your move?"},
]
outputs = pipe(
    messages,
    max_new_tokens=256,
)
print(outputs[0]["generated_text"][-1])


Prompt: You are an AI playing chess. You are playing as black, and your opponent is white. It is your turn. The board is as follows:
r n b q k b n r
p p p p p p p p
........
........
.... P...
........
P P P P. P P P
R N B Q K B N R
The sequence of moves so far is: [e2e4]
Select your next move (in UCI notation).
AI move: 
Player move: 
Prompt: You are an AI playing chess. You are playing as


In [ ]:
# ERROR_MOVE_PROMPT="Illegal move: {move}! Try again.\nYour move (in UCI notation) is:"
ERROR_MOVE_PROMPT="""Illegal move: {move}! Some legal moves are: [{legal_moves}]. Try again.
Your next move (in UCI notation) is:"""

SHOW_MAX_LEGAL_MOVES = 10
print(
  ERROR_MOVE_PROMPT.format(move="e2e4", board=board,
  legal_moves=', '.join([m.uci() for i, m in enumerate(board.legal_moves) if i >= (SHOW_MAX_LEGAL_MOVES-1)]))
  )

Illegal move: e2e4! Some legal moves are: [c7c6, b7b6, a7a6, h7h5, g7g5, f7f5, e7e5, d7d5, c7c5, b7b5, a7a5]. Try again.
Your move (in UCI notation) is:


In [ ]:
class LLMGame:
  def __init__(self, model, tokenizer, stockfish, base_prompt, color='white', stockfish_time_limit=None,
               show_legal_moves_limit=10,
               llm_illegal_move_retries=3, llm_illegal_move_prompt=ERROR_MOVE_PROMPT):
    self.model = model
    self.tokenizer = tokenizer
    self.stockfish = stockfish
    self.board = chess.Board()
    self.base_prompt = base_prompt
    self.color = color
    self.stockfish_time_limit = stockfish_time_limit
    self.show_legal_moves_limit = show_legal_moves_limit
    self.llm_illegal_move_retries = llm_illegal_move_retries
    self.llm_illegal_move_prompt = llm_illegal_move_prompt
    self.history = []
    self.outcome = None
    self.error_outcome = None

  def get_prompt(self, error_move=None):
    legal_moves = [m.uci() for i, m in enumerate(self.board.legal_moves) if i >= (self.show_legal_moves_limit-1)]
    prompt = self.base_prompt.format(
        llm_color=self.color,
        opponent_color='black' if self.color == 'white' else 'white',
        board=self.board,
        history=', '.join(self.history),
        legal_moves=', '.join(legal_moves)
    )
    if error_move:
      prompt += ('\n' + self.llm_illegal_move_prompt.format(
        move=error_move,
        board=self.board,
        legal_moves=', '.join(legal_moves)
      ))
    return prompt

  def make_engine_move(self):
    if self.board.is_game_over():
      return
    self.stockfish.set_fen_position(self.board.fen(), send_ucinewgame_token=True)
    if self.stockfish_time_limit:
      move = self.stockfish.get_best_move_time(self.stockfish_time_limit)
    else:
      move = self.stockfish.get_best_move()
    self.history.append(move)
    self.board.push_uci(move)

  def make_llm_move(self, retries=None):
    if self.board.is_game_over():
      return
    prompt = self.get_prompt()
    input_ids = tokenizer(prompt, return_tensors='pt').input_ids
    output = model.generate(input_ids, max_length=100)
    resp = tokenizer.decode(output[0])
    print(resp)
    move = resp.split('\n')[-1].strip()

    if retries is None:
      retries = self.llm_illegal_move_retries

    if move not in self.board.legal_moves:
      if retries > 0:
        prompt = self.get_prompt(error_move=move)
        self.llm_move(retries=retries-1)
      raise ValueError("Illegal move: {move}")

    self.history.append(move)
    self.board.push_uci(move)

  def play(self):
    self.board = chess.Board()
    self.history = []
    self.outcome = None
    self.error_outcome = None

    if self.color == 'black':
      if not self.board.is_game_over():
        try:
          self.make_engine_move()
          self.outcome = self.board.outcome()
        except ValueError as e:
          print(e)
          self.error_outcome = ('Engine', e)
          self.outcome = self.board.outcome()
          return

    while not self.board.is_game_over():
      if not self.board.is_game_over():
        try:
          self.make_llm_move()
          self.outcome = self.board.outcome()
        except ValueError as e:
          print(e)
          self.error_outcome = ('LLM', e)
          self.outcome = self.board.outcome()
          return
      if not self.board.is_game_over():
        try:
          self.make_engine_move()
          self.outcome = self.board.outcome()
        except ValueError as e:
          print(e)
          self.error_outcome = ('Engine', e)
          self.outcome = self.board.outcome()
          return